## Schedule Pipeline using AWS Events Bridge

Let us go ahead and schedule pipeline using AWS Events Bridge. We can take care of it either by using Lambda console or AWS Events Bridge console.
* To catch up we can schedule the job every 2 minutes and then we can change it to 15 minutes.
* As we are dealing with 45 days as baseline, the job should catch up with in couple of hours.
* We will also clean up every thing before scheduling the job.

In [ ]:
!aws s3 rm s3://itversitydata/messages --recursive

In [ ]:
import boto3

In [ ]:
dynamodb = boto3.resource('dynamodb')

In [ ]:
for table in dynamodb.tables.iterator():
    print(table)

In [ ]:
jobs_table = dynamodb.Table('gmail_jobs')
jobs_table.delete_item(Key={'job_id': 'gmail_jobs'})
item = {
    'job_id': 'gmail_ingest',
    'job_description': 'Ingest data from gmail to s3',
    'is_active': 'Y',
    'baseline_days': 45
}
jobs_table.put_item(Item=item)

In [ ]:
jrd_table = dynamodb.Table('gmail_job_run_details')

In [11]:
for item in jrd_table.scan()['Items']:
    jrd_table.delete_item(Key={'job_id': item['job_id'], 'job_run_time': item['job_run_time']})

* Here are the cron expressions to schedule every 2 minutes as well as every 15 minutes.

```
cron(0/2 * * * ? *)
cron(0/15 * * * ? *)
```

* Once the job is scheduled to run every 2 minutes to catch up we can validate to see if the emails from GMail are copied to s3 or not.

In [15]:
!aws s3 ls s3://itversitydata/messages/

2022-05-16 05:22:59    4981843 part-257533cd-d4aa-11ec-9629-7b9a7ee0bd59.json


In [13]:
jobs_table.scan()

{'Items': [{'job_description': 'Ingest data from gmail to s3',
   'is_active': 'Y',
   'job_id': 'gmail_ingest',
   'baseline_days': Decimal('45'),
   'job_run_bookmark_details': {'last_run_max_message_id': '17fe245c9e5f60c3',
    'last_run_start_time_epoch': Decimal('1648684800'),
    'last_run_end_time_epoch': Decimal('1648771200')}}],
 'Count': 1,
 'ScannedCount': 1,
 'ResponseMetadata': {'RequestId': 'JNNJGR4H1UCKF1JJUD0FMC8PHNVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 15 May 2022 23:53:00 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '352',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'JNNJGR4H1UCKF1JJUD0FMC8PHNVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3102346803'},
  'RetryAttempts': 0}}

In [16]:
jrd_table.scan()

{'Items': [{'job_id': 'gmail_ingest',
   'file_name': None,
   'job_run_bookmark_details': {'start_time_epoch': Decimal('1648684800'),
    'end_time_epoch': Decimal('1648771200'),
    'max_message_id': '17fe245c9e5f60c3'},
   'rows_processed': Decimal('77'),
   'job_run_time': Decimal('1652658769')}],
 'Count': 1,
 'ScannedCount': 1,
 'ResponseMetadata': {'RequestId': 'U7RLQPUJ5N5LHCNNAFH8K7H0CFVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 15 May 2022 23:53:22 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '309',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'U7RLQPUJ5N5LHCNNAFH8K7H0CFVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '3023659954'},
  'RetryAttempts': 0}}